# Structural Analysis Toolkit (SAT)

## If anything goes wrong:  
### Restart the kernel using the &#8635; button at the top of this notebook, or from the menu: `Kernel > Restart Kernel...`

## **During the tutorials just run through the cells using the given example files and code. Later you will use your own CAD drawing and edit the code to solve your own structural optimisation problem.**

### Flowchart for this Notebook:

![](SAT_flow.png)

* Green boxes show subroutines explained elsewhere...

## Loading the external files

### 0. Upload your DXF file (the example used here is called `example.dxf`), to the file browser on the left.
### 1. Run the cell below to load it into Python and create lists of `Node` and `Member` objects.

In [ ]:
# ===================================================
# Ensure the input filenames below match uploded files
# ===================================================
from import_functions import extract_structure_from_dxf

# input files
INPUT_DXF = "example.dxf"

# import data and create lists of nodes and members, each with its own ID, name, properties, etc...
nodes, members = extract_structure_from_dxf(INPUT_DXF, units='mm', tol=0.1)

##########################################
# Visualisation of the imported structure
##########################################
import visualisation_functions as viz

# show the imported structure
viz.visualize_truss(nodes, members, textscale=1, show_node_labels=True, show_member_labels=True)

#### &bullet; **Make sure all of the members and joints have unique labels,**
#####      $-$ if not you need to edit your CAD drawing to make it more accurate.

* We can inspect the nodes (and members) created:

In [ ]:
# list the nodes
print(nodes)

This is a dictionary with each of the nodes listed, lets look at one:
* click immediately after `.` and press the tab key to view attributes (properties) of the node object
* click on any of them (or type it) to view that property
e.g.:
```python
node.F_x
```

In [ ]:
node = nodes['C']
node.connectivity

### 2. Edit the dictionary in the cell below then run it to assign different joint types:

In [ ]:
##########################################
# Edit this to assign conditions to joints
##########################################

# comma-separated lists of node-labels as dictionary values 
# default value is "Pinned"
JOINT_CONDITIONS = {
"Pinned":            [],	     # Internal pin (free rotation, no support)	    ○	Blue
"Fixed":             [],         # Internal fixed joint (rigid)	                □	~Blue
"Reaction_Fixed":    [],	     # External fixed support	                    ■	Black
"Reaction_Pinned":   ['A'],      # External pinned support	                    ●	Black
"Reaction_Roller_H": ['E'],      # Horizontal roller (restrained vertically)	▲	Black
"Reaction_Roller_V": []	         # Vertical roller (restrained horizontally)	▼	Black
}

#########################
# Assign point conditions 
#########################
from struct_prop_funcs import apply_joint_conditions

# assign joint conditions
apply_joint_conditions(nodes, JOINT_CONDITIONS)

viz.visualize_truss(nodes, members, show_node_labels=True)

| Type                | Description                               | Marker | Colour |
| ------------------- | ----------------------------------------- | ------ | ----- |
| `Pinned`            | Internal pin (free rotation, no support)  | ○      | Blue  |
| `Fixed`             | Internal fixed joint (rigid)              | □      | Blue  |
| `Reaction_Fixed`    | External fixed support                    | ■      | Black |
| `Reaction_Pinned`   | External pinned support                   | ●      | Black |
| `Reaction_Roller_H` | Horizontal roller (restrained vertically) | ▲      | Black |
| `Reaction_Roller_V` | Vertical roller (restrained horizontally) | ▼      | Black |

### 3. Import the Materials list and properties from the `materials.xlsx` Excel file
* To edit your own: Download the `materials.xlsx` file (right-click in the file browser on the left) edit and re-upload (drag and drop into the file-browser or use the up arrow &uarr; above the file list). **Do not change the basic formatting of the excel sheet**, only add new rows or properties columns if needed.
### Run the cell below to import the file and display its contents


In [ ]:
from import_functions import read_materials_from_excel

MATERIAL_EXCEL = "materials.xlsx"  # path to your file

# import materials -> dataframe keyed by material name
material_props_df = read_materials_from_excel(MATERIAL_EXCEL, print_contents=False)
material_props_df

### 4.  Manual Configuration of Properties for `members`

### Edit the dictionaries in the cell below then run it to assign different joint and material properties:

In [ ]:
##########################################
# Edit this to assign materials to members 
#  * Note: names are very sensitive to number of spaces and upper/lower-case letters
##########################################

MANUAL_MATERIALS = {
#"114.3 x 10 CHS - S355": ["A-B", "B-E"],
"254x343x63 - T - S355": ["A-D", "B-D", "B-C"],
}

# any unassigned materials will get this one:
DEFAULT_MATERIAL = "100x100 C14"

##################
# Assign materials 
##################
from struct_prop_funcs import assign_materials_to_members
from printing_tools import member_dataframe


# assign materials: only names that exist in Excel will be assigned; others -> None
assign_materials_to_members(members, MANUAL_MATERIALS, material_props_df, default_material=DEFAULT_MATERIAL)

# Plot a figure of the new truss system
viz.visualize_truss(nodes, members, show_node_labels=True, show_member_labels=True, show_material_labels=True, textscale=0.8)

# show table and export
df = member_dataframe(members)

MEMBER_PROPERTY_FILE = "member_properties.xlsx"
# save member properties to excel
df.to_excel(MEMBER_PROPERTY_FILE)

## Next follows some examples of what you can do with your structural objects...

### 5. Check if Determinate

#### 2D truss diagnostics with automatic connectivity and total unknown reaction check.

![](determin_flow.png)

Prints a detailed report including:
- Determinacy check
- Support reaction analysis
- Potential mechanism nodes
- Total unknown reaction check

In [ ]:
n_nodes = len(nodes)
n_members = len(members)
n_reactions = sum(n.unknowns for n in nodes.values())
determinant = 2 * n_nodes - (n_members + n_reactions)

print("\n### Truss Diagnostics ###")
print(f"Number of nodes (joints):       {n_nodes}")
print(f"Number of members:             {n_members}")
print(f"Number of unknown reactions:   {n_reactions}")
print(f"Determinant (2j - (m+r)):     {determinant}")
# Overall determinacy status (truss) 
if determinant == 0:
    status = "=> Statically determinate (truss)"
elif determinant > 0:
    status = "!!! Unstable mechanism possible (truss)"
else:
    status = "!!! Statically indeterminate (truss)"
print(f"Status: {status}\n")

# Support analysis 
print("Supports and unknown reaction components:")
problem_supports = []
for label, n in nodes.items():
    cond = n.condition
    if "Reaction" in cond: 
        unknowns = n.unknowns
        print(f"  Node {label:>3}: {cond:20} -> {unknowns} unknown reaction(s)")
        if unknowns > 2:
            problem_supports.append((label, cond, "Too many unknown reactions → may cause indeterminacy"))

if problem_supports:
    print("\nProblematic Supports:")
    for label, cond, issue in problem_supports:
        print(f"  Node {label:>3}: {cond:20} -> {issue}")
else:
    print("\nNo problematic supports detected.")

# Total unknown reaction check for rigid 2D body
print("\nTotal unknown reaction check:")
if n_reactions > 3:
    print(f"!!! Total unknown reactions = {n_reactions} > 3 --> system is statically indeterminate for rigid 2D body")
elif n_reactions < 3:
    print(f"!!! Total unknown reactions = {n_reactions} < 3 --> system may be unstable (mechanism possible)")
else:
    print(f"=> Total unknown reactions = {n_reactions} --> statically determinate for rigid 2D body")

# Potential mechanism nodes
potential_mech_nodes = []
for label, n in nodes.items():
    cond = n.condition
    if "Reaction" in cond:
        continue  # skip supports
    n_conn = len(n.connectivity) #number of members from the node
    if n_conn < 2:
        potential_mech_nodes.append((label, n_conn, "Too few connected members --> possible local mechanism"))

if potential_mech_nodes:
    print("\nPotential Mechanism Nodes:")
    for label, n_conn, issue in potential_mech_nodes:
        print(f"  Node {label}: {n_conn} member(s) connected -> {issue}")
else:
    print("\nNo obvious local mechanism nodes detected.")



## Do Structural Calculations

### 6. Define the loads and run the cell below to perform load calculations

In [ ]:
# ==========================
#   LOAD ASSIGNMENT
# ==========================

P = 5
# Point load configuration
#nodes['C'].F_y = P # F_x is already zero
nodes['B'].F_y = 1

UDL = 2.5
wind = 6
# member loads
members['A-B'].F_y = UDL # 2.5 kN/m down UDL on member AB
members['B-E'].F_y = UDL # 2.5 kN/m down UDL on member AD
#members['C-E'].F_x = -wind # 2.5 kN/m down UDL on member AD

# ==============================
# PERFORM LOAD CALCULATIONS HERE
# ==============================
from struct_prop_funcs import calculate_total_forces
from structural_analysis import compute_support_reactions


# Apply loads
calculate_total_forces(nodes, members) 

reactions = compute_support_reactions(nodes, verbose=False)

# Visualize
viz.visualize_truss(nodes, members, textscale=0.9, show_reactions=True, plot_loads=True, show_load_labels=True, show_node_labels=True,  show_force_labels=True)

In [ ]:
print(reactions)

### 7. Check Equilbrium

In [ ]:
# Step 6: Detailed moment equilibrium check (debug)
print("\n ### Moment Equilibrium Breakdown (Clockwise Positive) ### ")
print(f"{'Label':<8} {'x':>8} {'y':>8} {'Fx':>10} {'Fy':>10} {'M = (xFy + yFx)':>18}")

# Applied loads (clockwise positive)
for lbl, n in nodes.items():
    Fx = n.F_xtotal or n.F_x #uses F_xtotal if it exists, F_x otherwise
    Fy = n.F_ytotal or n.F_y
    x, y = n.coords[:2]
    M = (x * Fy + y * Fx)  # negative for clockwise positive
    print(f"{lbl:<8} {x:8.3f} {y:8.3f} {Fx:10.3f} {Fy:10.3f} {M:18.6f}")

# Supports and reactions
support_labels = [lbl for lbl in reactions.keys()]
support_nodes = {lbl: nodes[lbl] for lbl in support_labels}

print(" ### Reactions (Clockwise Positive) ### ")
for lbl, r in reactions.items():
    s = support_nodes[lbl]
    x, y = s.coords[:2]
    M = (x * r['Ry'] + y * r['Rx'])  # clockwise positive
    print(f"{lbl:<8} {x:8.3f} {y:8.3f} {r['Rx']:10.3f} {r['Ry']:10.3f} {M:18.6f}")

# Compute totals (clockwise positive)
M_total_loads = sum(
    n.coords[0] * (n.F_ytotal or n.F_y)
    + n.coords[1] * (n.F_xtotal or n.F_x)
    for n in nodes.values()
)
M_total_reac = sum(
    support_nodes[l].coords[0] * r['Ry']
    + support_nodes[l].coords[1] * r['Rx']
    for l, r in reactions.items()
)
M_check = M_total_loads + M_total_reac

print(f"\nM_total (loads, CW+) = {M_total_loads:.6f}")
print(f"M_reactions (CW+)   = {M_total_reac:.6f}")
print(f"ΣM (CW+) = {M_check:.6f}  (should be ≈ 0)\n")



### 8. Use method of joints to solve axial forces

In [ ]:
from structural_analysis import solve_truss_joint_iteration


# STRUCTURAL ANALYSIS
member_forces = solve_truss_joint_iteration(nodes, members)

# Visualize everything
viz.visualize_truss(nodes, members, textscale=0.9, show_reactions=True, plot_loads=True, show_node_labels=True, show_load_labels=True, show_forces=True, show_force_labels=True)

### 9. Compute and visualise member bending moments

In [ ]:
from structural_analysis import compute_member_bending_moments

member_moments = compute_member_bending_moments(members)

viz.visualize_truss(nodes, members, 
                    textscale=0.9, 
                    show_reactions=True, 
                    plot_loads=True, 
                    show_load_labels=True, 
                    show_node_labels=True,
                    show_bending_moments=True, 
                    show_bending_moment_labels=True, 
                    moment_scale=0.1)

In [ ]:
members['B-E'].moments

## Next Steps... Do interesting things!

### 10. Calculates Stresses:

We need to account for the plus/minus effect:

σ total= σ axial ± σ bending

Tension axial stress + bending at the top fiber
Compression axial stress + bending at the top fiber


![](exercise_flow.png)

In [ ]:
### Compute combined stresses (axial + bending) for all members.
### =======
### EXERCISE!!!
### ==========
import numpy as np 

def calculate_stresses(members):
    ###
    # YOUR CODE HERE
    ###
    return max_stress, stresses

max_stress,stresses = calculate_stresses(members)

viz.visualize_truss(nodes, members, textscale=0.9, show_reactions=True, plot_loads=True, show_node_labels=True, show_load_labels=True, show_stresses=True, show_colorbar=True )


In [ ]:
m = members['A-B']
m.stress

### 11. Compute utilisation factors for members based on stress and material properties.

In [ ]:
import math

def compute_utilisations(members):
    warn_limit=1.0
    utilisations = {}
    max_utilisation = 0.0
    overstressed = []
    
    # ANSI color helpers
    RED = "\033[91m"
    RESET = "\033[0m"
    
    print("\n=== Utilisation Summary ===")
    print(f"{'Member':<10} {'σ_total (MPa)':>15} {'Allow (MPa)':>15} {'Type':>12} {'Utilisation':>15}")
    print("-" * 75)
    
    for m in members.values():
        sigma = m.stress
        props = m.properties
    
        ft = props["ft"] 
        fc = props["fc"] 
    
        # Decide allowable stress based on sign
        if sigma >= 0:
            allowable = ft 
            stress_type = "Tension"
        else:
            allowable = fc 
            stress_type = "Compression"
    
        util = abs(sigma) / allowable
    
        utilisations[m.name] = util
        m.utilisation=util
        if not math.isinf(util):
            max_utilisation = max(max_utilisation, util)
    
        # Highlight overstressed members
        over = util > warn_limit
        allow_str = f"{allowable:.3f}" 
        util_str = f"{util:.3f}" 
    
        line = f"{m.name:<10} {sigma:15.3f} {allow_str:>15} {stress_type:>12} {util_str:>15}"
        if over:
            line = RED + line + RESET
            overstressed.append((m.name, util))
    
        print(line)
    
    print("-" * 75)
    print(f"Max utilisation = {max_utilisation:.3f}")
    
    if overstressed:
        print(f"\n!!!  Overstressed members (util > {warn_limit}):")
        for name, util in overstressed:
            print(f"  - {name}: {util:.3f}")
    else:
        print("\n=> All members within allowable limits.")
    return utilisations


utils1 = compute_utilisations(members)

viz.visualize_truss(nodes, members, textscale=0.9, show_reactions=True, plot_loads=True, show_load_labels=True, show_utilisations=True, show_colorbar=True )


## Making modifications to the structure

### 12. Run this to see and example of a structural change:

In [ ]:
# Factor to stretch the structure
#WIDTH_SCALE = 2.5
HEIGHT_SCALE = 2.5

from copy import deepcopy
# deepcopy also makes a copy of all the objects within the dictionary
nodes2 = deepcopy(nodes)
members2 = deepcopy(members)
for m in members2.values():
    m.nodes=nodes2 

# scale and visualize
for n in nodes2.values():
    n.coords[1] *= HEIGHT_SCALE #same as: n.coords[1] = n.coords[1]*HEIGHT_SCALE

# Visualize
viz.visualize_truss(nodes, members, textscale=0.9, show_reactions=True, plot_loads=True, show_load_labels=True, show_node_labels=True)
viz.visualize_truss(nodes2, members2, textscale=0.9, show_reactions=True, plot_loads=True, show_load_labels=True, show_node_labels=True)

In [ ]:
from computation_tools import compute_node_angles
from printing_tools import comparison_data

# Member lengths 
def member_lengths(members):
    lengths = {}
    for m in members.values():
        L = m.compute_length()
        lengths[m.name] = L
    return lengths


# Store original angles and lengths
original_lengths = member_lengths(members)
original_angles = compute_node_angles(nodes, members)


# calculate new angles and lengths
new_lengths = member_lengths(members2)
new_angles = compute_node_angles(nodes2, members2)

# Show the values as a table
df = comparison_data(original_angles, new_angles, original_lengths, new_lengths, print_output=False)

COMPARISON_SHEET_FILENAME = "compare_angles.xlsx"

# save data to excel
df.to_excel(COMPARISON_SHEET_FILENAME)
df

In [ ]:
# STRUCTURAL RE-ANALYSIS

reactions2 = compute_support_reactions(nodes2, members2)
member_forces2 = solve_truss_joint_iteration(nodes2, members2)
member_moments2 = compute_member_bending_moments(members2)
max_stress2,stresses2 = calculate_stresses(members2)
utils2 = compute_utilisations(members2)

# Plot a figure of the new truss system
viz.visualize_truss(nodes2, members2, show_reactions=True, plot_loads=True, show_node_labels=True, show_forces=True, show_bending_moments=True, show_bending_moment_labels=False, moment_scale=0.1)
viz.visualize_truss(nodes2, members2, textscale=0.9, show_reactions=True, plot_loads=True, show_load_labels=True, show_utilisations=True, show_colorbar=True )


## 13. Finally: Save your new structure back to a DXF to import into AutoCAD

In [ ]:
from import_functions import export_to_dxf

OUTPUT_DXF = "scaled_structure.dxf"

# save new structure
export_to_dxf(nodes, members, OUTPUT_DXF, units='mm')